<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Neural Signals and Signal Processing (NX-421)</h2>
<hr style="clear:both"></hr>

Welcome to the laboratory computers for the course "Neural signals and signal processing". 
This week, we finish up the preprocessing of fMRI on some advanced points, which you might need when working on your mini-project.
We will then look at functional Near Infrared Spectroscopy (fNRIS).

In [ ]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ,get_json_from_file

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

from IPython.display import display, HTML


NOTE: Do not worry if you get the message "Gtk-Message: 09:26:08.207: Failed to load module "canberra-gtk-module", the Notebook will still work!

In [ ]:

# Inject custom CSS
custom_css ="""
<style>

  .fit {
    object-fit: cover;
  }

  .container {
    display: flex;
    align-items: center; /* Align blocks vertically in the middle */
    justify-content: flex-start; /* Align blocks to the left */
  }

  .imageDiv {
      background: #fff;
      display: block;height: 150px;width: 150px;padding: 10px;border-radius: 2px;box-shadow: 0 1px 4px rgba(0, 0, 0, 0.3), 0 0 40px rgba(0, 0, 0, 0.1) inset;flex-shrink: 0;
  }
    .arrow-right {
      width: 0; 
      height: 0; 
      border-top: 1em solid transparent;
      border-bottom: 1em solid transparent;
      border-left: 1em solid #000;
    }
      .col-md-10 {
        display: flex;
        align-items: center;
      }
    
    .bby {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        font-family: Arial, sans-serif;
    }
    
    .flow-container {
        display: flex;
        align-items: center;
        justify-content:center;
    }
    
    .box-text-container {
        flex-direction: column;
        display: center;
        align-items: center;
        justify-content:center;
    }
    
    .step-box {
        background-color: lightgray;
        padding: 0.5em;
        margin: 0 0.5em;
        text-align: center;
        font-weight: bold;
        border-radius: 0.25em;
        border: 0.15em solid black;
        min-width: 6em;
    }
    
    .arrow {
        font-size: 1.5em;
        color: blue;
        font-weight: bold;
        margin: 0 0 0.5em;
    }
    
    .text {
        font-weight: bold;
        font-size: 0.9em;
        margin: 0 0.5em;
    }
    
    .box-wrapper {
        display: flex;
        align-items: center;
        padding: 0.5em;
        border: 0.15em solid black;
        border-radius: 10px;
    }
    
    /* Style for the text below the box */
    .kapt_2 {
        margin-top: 0.5em;
        font-size: 1em;
        font-weight: bold;
    }

    .disp_p {
        font-size:2.5em;
    }
"""

display(HTML(custom_css))

# Part 2: Get acquainted with fNIRS data preprocessing

In contrast to functional magnetic resonance imaging (fMRI), functional Near-Infrared Spectroscopy (fNIRS) distinguishes itself with portability and adaptability, making it especially suitable for research involving intricate populations like infants, tasks characterized by motion, and real-world settings.

Nevertheless, it is important to acknowledge that collecting fNIRS data requires rigorous preprocessing. This requirement arises from fNIRS's susceptibility to 1) superficial physiological interferences, such as those stemming from scalp blood flow and 2) motion artifacts, especially those resulting from sensor displacement.

In [ ]:
import os.path as op
from mne.preprocessing.nirs import temporal_derivative_distribution_repair

## 2.1 Data set

### 2.1.1 Description

We will work on a set of hemodynamic data measured on one subject during a finger tapping paradigm with three conditions: 1) Tapping the left thumb to fingers, 2) Tapping the right thumb to fingers and 3) A control when nothing happens. Each tapping lasts 5 seconds and there are 30 trials in each condition.The recording was performed using fNIRS sensors located over motor areas of the cortex. 

Data were provided by Luke, R., & McAlpine, D. (2021). fNIRS Finger Tapping Data in BIDS Format (Version v0.0.1) (https://doi.org/10.5281/zenodo.5529797),

### 2.1.2 Loading

Load the dataset by running the next cell.

In [ ]:
import mne
import os.path as op
#Download dataset
fnirs_data_folder = mne.datasets.fnirs_motor.data_path(path='/home/jovyan/Data/mne_data/')
#Get the path for the dataset folder 
fnirs_data_folder=op.join(fnirs_data_folder, 'Participant-1')
#Load the dataset 
raw_intensity = mne.io.read_raw_nirx(fnirs_data_folder, verbose=True, preload=True)

### 2.1.3 Recording setting

Display and read information on the recording setting such as the number of channels, the file duration and the sampling frequency by runing the next cell. 

In [ ]:
#Display recording setting
raw_intensity

In [ ]:
interactive_MCQ(4,6)

The placement of fNIRS sensors holds significance for achieving **a good spatial resolution** and an **accurate sensor placement**.

Let's take a look at the locations of sensors.

In [ ]:
!pip install "pyvista[jupyter]"
!pip install pyvistaqt
!pip install pyqt5
!pip install qtconsole pyqt5

In [ ]:
%matplotlib qt5
subjects_dir = op.join(mne.datasets.sample.data_path('/home/jovyan/Data/mne_data'), "subjects")

brain = mne.viz.Brain(
subject="fsaverage", subjects_dir=subjects_dir, background="black", cortex="0.5"
)
brain.add_sensors(
    raw_intensity.info,
    trans="fsaverage",
    fnirs=["channels", "pairs", "sources", "detectors"],
)
brain.show_view(azimuth=20, elevation=60, distance=400)

<p style="color:green"> See below the expected 3D visualization of the fNIRS sensors placed over the head: </p>

<img src="imgs/3Dmap.png">

In the 3D visualization, we represent source-detector pairs or channels as lines. 

<span style="color:blue; text-decoration:underline;">Multiple Choice Question</span>

*What is the cortical region covered by this measurement ?*
1. Motor cortex
2. Visual cortex

   <p style="color:green"> In this finger tapping task, sensors are placed over the motor cortex. This region is located in the dorsal portion of the frontal lobe. </p>


### 2.1.4 Experimental design

Let's now have a look at the experimental design used.

The experimental designs most commonly employed by auditory fNIRS researchers are block- and event-related designs. In an event related design, each task is presented individually for a short amount of time e.g., 3 seconds. In this way, tasks can be more  randomized,  rather  than  being  blocked  together  by  condition. Conversely, in a block-related design, blocks of tasks, each lasting at least 10 seconds, recur before intervals of rest. 

The choice of the experimental design depends on a range of factors, including the statistical power, the duration of the experiment, and whether the design provides the flexibility to study the effect of interest. While the block design might lead to higher detection power, it can also induce learning and boredom effects which may bias the results. On the other hand, event-related designs reduce the effects of learning, boredom while exhibiting loss in detection power. 

<span style="color:blue; text-decoration:underline;">Open question</span>

*What are the analytical challenges associated with employing a block-related experimental design featuring continuous stimulation and brief rest blocks?*

   <p style="color:green"> In a block-related experimental design with continuous stimulation and brief rest blocks, we can end-up having a non-linear summation of the hemodynamic responses to stimuli. This non-linearity can cause a mismatch between the predicted  summation of responses obtained with linear approaches (e.g., averaging or general linear modelling) and the actual observed data. </p>







Run the next cell to display the sequence of events used in this experiment.

In [ ]:
#Load the event annotations
events, event_dict=mne.events_from_annotations(raw_intensity,verbose=False)
#Assign each label to the event (based on recording setting)
event_dict={'Control':1,'Tapping/Left':4,'Tapping/Right':3,'ExperimentEnds':2}
#Display the sequence of events 
plt.rcParams["figure.figsize"]=(10,6)
mne.viz.plot_events(events,event_id=event_dict,sfreq=raw_intensity.info['sfreq']);

As you can see, there is a significant gap or interstimulus interval between each individual stimulus. We can thus conclude that an event-related design was employed in this experiment.

Now that we have gained an understanding of the experiment and recording processes, we can start the visualization of the raw data collected. To proceed, run the next cell.

In [ ]:
#Create a figure window
plt.rcParams["figure.figsize"]=(40,40)
#Plot time-series of raw light intensity data 
mne.viz.plot_raw(raw_intensity,start=120,duration=80,n_channels=56,show_scrollbars=False)

In this plot, we can observe the time-series of the data collected, represented as the change in light intensity resulting from light absorption at specific wavelengths. Oxygenated hemoglobin (HbO) absorbs light at "850"nm, while deoxygenated hemoglobin (HbR) absorbs light at "760"nm. Each channel comprises a pair consisting of a source (labeled as "S" in the plot) and a detector (labeled as "D" in the plot). 

Here, we are showing all the channels (y-axis) but, for clarity, we are only focusing on a specific timeframe, spanning from 120 to 200 seconds (x-axis). If you wish to customize the visualization parameters, please refer to the documentation of [mne.viz.plot_raw](https://mne.tools/stable/generated/mne.viz.plot_raw.html).

<span style="color:blue; text-decoration:underline;">Open question</span>

*Visually, are you able to localize a motion artifact?*

   <p style="color:green"> There are two types of motion artefacts we can visualize in this graph: a spike between 185-190s and a baseline shift in the S8_D8 850 channel. These two types of artefact can easily happen in motor tasks. </p>

## 2.2 Preprocess the data

A typical preprocessing pipeline for fNIRS data includes the following key steps:

<hr>

*Step 1: Raw Intensity to Optical Density Conversion*

<hr> 

*Step 2: Motion Artifact Removal*

<p style='margin-top:1em;'>💡 This step is more effective when performed at the beginning of the pipeline to prevent the propagation of errors across wavelengths and minimizes cross talk between signals obtained at different wavelengths.

<hr> 

*Step 3: Optical Density to Concentration Conversion*

<hr>

*Step 4: Physiological Oscillation Filtering*

<p style='margin-top:1em;'>💡 This step is more effective when performed after the conversion step using the modified Beer-Lambert law, as physiological sources of error impact HbO and HbR in a manner consistent with the Beer-Lambert equation.
    
    
<hr>

### 2.2.1 Converting raw intensity data to optical density

The first step is to convert the changea in light intensity into changes in optical density using the modified Beer-Lambert law (Eq 1). 

\begin{equation}
OD_\lambda - OD_{R\lambda} = log\frac{I_o}{I} (1)
\end{equation}

With:
- $OD_\lambda$ the optical density of the medium for a given wavelength $\lambda$
- $OD_{R\lambda}$ the optical density of light scattering within human tissue
- $I_o$ the incident light intensity 
- $I$ the transmitted light intensity 

Run the next cell to convert raw intensity into optical density.

In [ ]:
#Convert raw intensity into optical density data
raw_od=mne.preprocessing.nirs.optical_density(raw_intensity)

### 2.2.2 Correcting motion artefacts

In fNIRS data, two types of artifacts are commonly encountered: baseline shifts (indicating sensor displacement without returning to the initial position) and spike artifacts (characterized by sensor oscillations). As you correctly guessed earlier, the abrupt shift occurring around ~190 seconds in the time-series plot above corresponds to a spike artifact.

Here, we will employ a technique known as Temporal Derivative Distribution Repair (TDDR), which is designed to address motion artefacts. TDDR uses the temporal derivative of the signal to correct the signal. 

For a more comprehensive understanding of the TDDR method and its application, you can refer to the following paper:
- Frank A. Fishburn, Ruth S. Ludlum, Chandan J. Vaidya, and Andrei V. Medvedev. "Temporal Derivative Distribution Repair (TDDR): A Motion Correction Method for fNIRS." NeuroImage, 184:171–179, 2019. doi:10.1016/j.neuroimage.2018.09.025.

Run the cell below to apply the TDDR on the optical density data. 

In [ ]:
#Apply TDDR on optical density data
raw_od_preproc=temporal_derivative_distribution_repair(raw_od)

Execute the following cell to visualize the optical density data after applying TDDR. 

In [ ]:
#Create a figure window
plt.rcParams["figure.figsize"]=(40,40)
#Plot time-series of optical density data following TDDR application
mne.viz.plot_raw(raw_od_preproc,show_scrollbars=False,start=120,n_channels=56,duration=80)

<span style="color:blue; text-decoration:underline;">Open question</span>

*From a visual standpoint, can we conclude that TDDR successfully eliminated the motion artifacts from your data?*

 <p style="color:green"> While the TDDR corrected the baseline shift seen in some of the channels (e.g., S8_D8 850) it did not remove the spikes from the signal. This low sensitivity to motion-related spikes is actually one of the limits of the TDDR method. More robust methods that are however not proposed in mne-nirs library are the wavelet filtering or the spline interpolation.                                                                                                               For a comparison of motion artefact correction techniques, you can refer to the following paper:
    Cooper, R. J., Selb, J., Gagnon, L., Phillip, D., Schytz, H. W., Iversen, H. K., ... & Boas, D. A. (2012). A systematic comparison of motion artifact correction techniques for functional near-infrared spectroscopy. Frontiers in neuroscience, 6, 147.</p>



### 2.2.3 Converting optical density to concentration 

The changes in optical density are then converted into changes of concentration using the modified beer lambert law (Eq 1): 

\begin{equation}
\Delta c = \frac{\Delta OD_\lambda }{\epsilon_\lambda lB} 
\end{equation}

Whith:
- $\Delta$ c the molar concentration change (in M)
- $\epsilon_\lambda$ the moral absorption coefficient (in $M^-1 cm^-1$) for a given wavelength $\lambda$
- l the the optical pathlength 
- B the pathlength correction factor

For more information on the equation, see the reference: Delpy DT, Cope M, Zee P van der, Arridge S, Wray S, Wyatt J. Estimation of optical pathlength through tissue from direct time of flight measurements. Phys Med Biol 1988; 33: 1433-1442.

Run the cell below to compute the concentration changes.

In [ ]:
#Import required beer-lambert law related function
from mne.preprocessing.nirs import beer_lambert_law
#Convert optical density into concentration change (hemodnyamic data)
raw_haemo=beer_lambert_law(raw_od_preproc)

### 2.2.4 Filtering out physiological oscillations

Let's now filter out physiological oscillations.

To begin, we'll assess the extent of physiological oscillations within the data using power spectral density (PSD) analysis. Based on the sequence of events outlined previously, we note that stimuli for a same condition (e.g., tapping right) are presented at a rate of approximately once every 1/100 seconds. Consequently, we should anticipate a hemodynamic response to this specific stimulus occurring at this particular frequency. Any additional components identified in the PSD analysis can be attributed to other forms of oscillations, including physiological ones.

Run the following cell to initiate the signal decomposition process.

In [ ]:
#Compute the PSD of non-filtered hemodynamic data 
fig = spectrum = raw_haemo.compute_psd().plot(average = True)
fig.suptitle('Before filtering', weight='bold', size='x-large')
fig.subplots_adjust(top=0.88)
plt.show()

The PSD shows a very low peak which may correspond to the hemodynamic responses to task stimulations. On the other hand, the presence of a frequency peak at 1.25 Hz aligns with the heart rate. 

Based on the information provided, apply a low-pass filter to retain only the hemodynamic responses. You may need to utilize the filter method available in the mne.io.Raw class as outlined in the MNE documentation (https://mne.tools/stable/generated/mne.io.Raw.html).

<p style="color:green"> MNE library provides two methods for filtering raw data: filter_data() function or the method filter. Here, we will utilize the filter() method, as it is more user-friendly and integrates seamlessly with subsequent analysis steps. 

<p style="color:green"> In the context of event-related designs, the goal is to extract hemodynamic responses to specific events. In our case, we aim to compare the hemodynamic responses to different events. 
Therefore, we have three distinct signals of interest: hemodynamic responses to 1) tapping left, 2) tapping right, and 3) control events. Maintaining the stimulation frequency in these signals is crucial. To determine this stimulation frequency, 
refer to the event sequence above. You'll notice there is around 1 stimulation per 100 seconds for each event. 
In the power spectrum above, you can identify a peak at approximately 0.01Hz, likely associated to this event-related hemodynamic response. You may also want to get rid of physiological oscillations: among the most important, there is the heart rate at ~1Hz and the breathing rate at ~0.3Hz. You can see the corresponding peaks in the PSD above. Based on this, the upper pass-band edge of your low pass filter can be set to 0.2Hz. You can use transbandwidth to make a smooth transition between the passband (0.2Hz) and the stopband (0.3Hz, frequency above which you will get your signal contaminated by physiological oscillations). Filters with narrow transition bandwidths are often preferred in applications where closely spaced frequency components need to be separated such as ours. However, be aware that you cannot narrow too much the transition bandwidth with low-order filters as the ones proposed by MNE library.</p>

In [ ]:
###########
# Now apply the filter method to retain only the hemodynamic responses from your signal
##########

#And now, let's try to filter the hemodynamic data using the filter method for raw objects 
raw_haemofiltered=raw_haemo.filter(0, 0.2, h_trans_bandwidth=0.1,
                             l_trans_bandwidth=0)


<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'> Whenever you apply a filter, you should check that you are not removing the signal of interest. For that, make sure the task stimulation frequency is not within the frequency range of your filter ! </p>
</span>
</div>

Run the next cell to plot the PSD of your filtered data.

In [ ]:
#Plot the spectrum of filtered hemodynamic data 
fig = spectrum = raw_haemofiltered.compute_psd().plot(average = True)
fig.suptitle('After filtering', weight='bold', size='x-large',y=1.1)
fig.subplots_adjust(top=0.88)
plt.show()

Run the next cell to visualize the filtered data.

In [ ]:
#Plot time-series of hemodynamic data obtained with one channel 
raw_haemofiltered.plot(start=0,duration=200,n_channels=1,show_scrollbars=False)

Congratulations ! You now have all the basics to understand and preprocess fNIRS data!  

<div class="alert alert-success">
<p><b>🎉 You've reached the end of this week's notebook! Congratulations! 🎉 </b></p>
</div>